# GEO877 Project    
**Authors:** *Alessandro Joshua Pierro 20-733-861, Alessandro Jud, Andreas Hvidt, Nina Moffat, Saskia Stierli*
**Date:** *2024-06-7*   
## Necessary Libraries

In [5]:
import json
import numpy as np

## Class Definitions

In [6]:
class Point():
    def __init__(self, x=None, y=None):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return f'Point(x={self.x:.2f}, y={self.y:.2f})'
    
    def __eq__(self, other): 
        if not isinstance(other, Point):
            return NotImplemented
        return self.x == other.x and self.y == other.y

    def __hash__(self):
        return hash((self.x, self.y))
    
    def distEuclidean(self, other):
        return np.sqrt((self.x-other.x)**2 + (self.y-other.y)**2)
    
    def distManhattan(self, other):
        return abs(self.x-other.x) + abs(self.y-other.y)

    def distHaversine(self, other):
        r = 6371000
        phi1 = np.radians(self.y)
        phi2 = np.radians(other.y)
        lam1 = np.radians(self.x)
        lam2 = np.radians(other.x)
        d = 2 * r * np.arcsin(np.sqrt(np.sin((phi2 - phi1)/2)**2 + 
                                      np.cos(phi1) * np.cos(phi2) * np.sin((lam2 - lam1)/2)**2))
        return d   
    
    def __det(self, p1, p2):
        det = (self.x-p1.x)*(p2.y-p1.y)-(p2.x-p1.x)*(self.y-p1.y)       
        return det

    def leftRight(self, p1, p2):
        side = self.__det(p1, p2)
        if side != 0:
            side = side/abs(side)
        return side

class Segment():    
    def __init__(self, p0, p1):
        self.start = p0
        self.end = p1
        self.length = p0.distEuclidean(p1)
    
    def __repr__(self):
        return f'Segment with start {self.start} and end {self.end}.' 

    def __eq__(self, other): 
        if (self.start == other.start or self.start == other.end) and (self.end == other.end or self.end == other.start):
            return True
        else:
            return False
    
    def __hash__(self):
        return hash((self.start.x, self.start.y, self.end.x, self.end.y))    
    
    def intersects(self, other):        
        self_bbox = Bbox(self)
        other_bbox = Bbox(other)
        bbox_overlap = self_bbox.intersects(other_bbox)
        if bbox_overlap == False:
            return False
        else:
            apq = self.start.leftRight(other.start, other.end)
            bpq = self.end.leftRight(other.start, other.end)
            pab = other.start.leftRight(self.start, self.end)
            qab = other.end.leftRight(self.start, self.end)           
            if (apq + bpq == 0 and pab + qab == 0): 
                return True
            else:
                return False      

class Bbox():    
    def __init__(self, data):
        if isinstance(data, Segment):
            x = [data.start.x, data.end.x]
            y = [data.start.y, data.end.y]
        else:      
            x = [i.x for i in data]
            y = [i.y for i in data]
        self.ll = Point(min(x), min(y))
        self.ur = Point(max(x), max(y))
        self.ctr = Point((max(x)-min(x))/2, (max(y)-min(y))/2)
        self.area = (abs(max(x)-min(x)))*abs((max(y)-min(y)))    
           
    def __repr__(self):
        return f'Bounding box with lower-left {self.ll} and upper-right {self.ur}' 
    
    def __eq__(self, other): 
        if (self.ll == other.ll and self.ur == other.ur):
            return True
        else:
            return False
    
    def __hash__(self):
        return hash(self.ll, self.ur)  
    
    def intersects(self, other):       
        if (self.ur.x > other.ll.x and other.ur.x > self.ll.x and
            self.ur.y > other.ll.y and other.ur.y > self.ll.y):
            return True
        else:
            return False
        
    def contains(self, other):
        if (self.ur.x >= other.ur.x and self.ll.x <= other.ll.x and
            self.ur.y >= other.ur.y and self.ll.y <= other.ll.y):
            return True
        else:
            return False
        
    def intersectsRegion(self, other):
        if self == other:
            return self
        elif self.contains(other):
            return other
        elif self.intersects(other):
            llx = max(self.ll.x, other.ll.x)
            lly = max(self.ll.y, other.ll.y)
            urx = min(self.ur.x, other.ur.x)
            ury = min(self.ur.y, other.ur.y)
            return Bbox([Point(llx, lly), Point(urx, ury)])
        else:
            return None
        
    def containsPoint(self, p):
        if (self.ur.x >= p.x and p.x >= self.ll.x and
            self.ur.y >= p.y and p.y >= self.ll.y):
            return True
        return False

class PointGroup(): 
    def __init__(self, data=None, xcol=None, ycol=None):
        self.points = []
        self.size = len(data)
        for d in data:
            self.points.append(Point(d[xcol], d[ycol]))
    
    def __repr__(self):
        return f'PointGroup containing {self.size} points' 

    def __getitem__(self, key):
        return self.points[key]

class Polygon(PointGroup):  
    def __init__(self, data=None, xcol=None, ycol=None):
        self.points = []
        self.size = len(data)
        for d in data:
            self.points.append(Point(d[xcol], d[ycol]))
        self.bbox = Bbox(self)
        
    def __repr__(self):
        return f'Polygon PointGroup containing {self.size} points' 
  
    def isClosed(self):
        start = self.points[0]
        end = self.points[-1]
        return start == end

    def removeDuplicates(self):
        oldn = len(self.points)
        self.points = list(dict.fromkeys(self.points))
        self.points.append(self.points[0])
        n = len(self.points)
        print(f'The old polygon had {oldn} points, now we only have {n}.')
    
    def __signedArea(self):
        a = 0
        xmean = 0
        ymean = 0
        for i in range(0, self.size-1):
            ai = self[i].x * self[i+1].y - self[i+1].x * self[i].y
            a += ai
            xmean += (self[i+1].x + self[i].x) * ai
            ymean += (self[i+1].y + self[i].y) * ai
        a = a/2.0
        return a, xmean, ymean
    
    def area(self):
        a, xmean, ymean = self.__signedArea()
        area = abs(a)
        return area

    def centre(self):
        a, xmean, ymean = self.__signedArea()
        centre = Point(xmean/(6*a), ymean/(6*a))
        return centre

    def containsPoint(self, p):
        if (self.bbox.containsPoint(p) == False):
            return False
        ray = Segment(p, Point(self.bbox.ur.x+1, p.y))
        count = 0
        for i in range(0, self.size-1):
            start = self[i]
            end = self[i+1]
            if (start.y != end.y): 
                if ((p.x < start.x) and (p.y == start.y)): 
                    count = count + 1
                else:
                    s = Segment(start, end)
                    if s.intersects(ray):
                        count = count + 1
        if (count%2 == 0):
            return False
        return True

    def get_points(self):
        return self.points
    
    def merge_poly(self, poly2):
        list_points = []
        for p in self.points:
            list_points.append([p.x, p.y])
        for p in poly2.points:
            list_points.append([p.x, p.y])
        merged_polygon = Polygon(data = list_points, xcol = 0, ycol = 1)
        merged_polygon.removeDuplicates()
        return merged_polygon

## Functions to Load Data

In [7]:
def load_points_from_geojson(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    points = [Point(feature['geometry']['coordinates'][0], feature['geometry']['coordinates'][1])
              for feature in data['features']]
    return points

def load_segments_from_geojson(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    segments = []
    for feature in data['features']:
        coords = feature['geometry']['coordinates']
        if isinstance(coords[0], list):
            if isinstance(coords[0][0], list):
                for subcoords in coords:
                    for i in range(len(subcoords) - 1):
                        p1 = Point(subcoords[i][0], subcoords[i][1])
                        p2 = Point(subcoords[i + 1][0], subcoords[i + 1][1])
                        segments.append(Segment(p1, p2))
            else:
                for i in range(len(coords) - 1):
                    p1 = Point(coords[i][0], coords[i][1])
                    p2 = Point(coords[i + 1][0], coords[i + 1][1])
                    segments.append(Segment(p1, p2))
        else:
            p1 = Point(coords[0], coords[1])
            p2 = Point(coords[2], coords[3])
            segments.append(Segment(p1, p2))
    return segments

def load_polygons_from_geojson(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    polygons = []
    for feature in data['features']:
        coords = feature['geometry']['coordinates'][0]
        polygon_points = [[c[0], c[1]] for c in coords]
        polygons.append(Polygon(data=polygon_points, xcol=0, ycol=1))
    return polygons

## Data Loading

In [8]:
# Set base data path (one level up from the current working directory)
base_data_path = "../data/geojson/"

# Define paths for Zurich and Graubunden, Swiss Topo and OSM data
zurich_swisstopo_path = base_data_path + "zurich/swisstopo/"
graubunden_swisstopo_path = base_data_path + "graubunden/swisstopo/"

zurich_osm_path = base_data_path + "zurich/osm/"
graubunden_osm_path = base_data_path + "graubunden/osm/"

# Load Swiss Topo geojsons for Zurich
motorway_zh_swisstopo_segments = load_segments_from_geojson(zurich_swisstopo_path + "motorway_zh_swisstopo.geojson")
mainroads_zh_swisstopo_segments = load_segments_from_geojson(zurich_swisstopo_path + "mainroads_zh_swisstopo.geojson")
smallpaths_zh_swisstopo_segments = load_segments_from_geojson(zurich_swisstopo_path + "smallpaths_zh_swisstopo.geojson")
print("Loaded Swiss Topo data for Zurich")

# Load Swiss Topo geojsons for Graubunden
motorway_gr_swisstopo_segments = load_segments_from_geojson(graubunden_swisstopo_path + "motorway_gr_swisstopo.geojson")
mainroads_gr_swisstopo_segments = load_segments_from_geojson(graubunden_swisstopo_path + "mainroads_gr_swisstopo.geojson")
smallpaths_gr_swisstopo_segments = load_segments_from_geojson(graubunden_swisstopo_path + "smallpaths_gr_swisstopo.geojson")
print("Loaded Swiss Topo data for Graubunden")

# Load OSM geojsons for Zurich
motorway_zh_osm_segments = load_segments_from_geojson(zurich_osm_path + "motorway_zh_osm.geojson")
mainroads_without_zh_osm_segments = load_segments_from_geojson(zurich_osm_path + "mainroads_without_zh_osm.geojson")
mainroads_with_zh_osm_segments = load_segments_from_geojson(zurich_osm_path + "mainroads_with_zh_osm.geojson")
smallpaths_without_zh_osm_segments = load_segments_from_geojson(zurich_osm_path + "smallpaths_without_zh_osm.geojson")
smallpaths_with_zh_osm_segments = load_segments_from_geojson(zurich_osm_path + "smallpaths_with_zh_osm.geojson")
print("Loaded OSM data for Zurich")

# Load OSM geojsons for Graubunden
motorway_gr_osm_segments = load_segments_from_geojson(graubunden_osm_path + "motorway_gr_osm.geojson")
mainroads_without_gr_osm_segments = load_segments_from_geojson(graubunden_osm_path + "mainroads_without_gr_osm.geojson")
mainroads_with_gr_osm_segments = load_segments_from_geojson(graubunden_osm_path + "mainroads_with_gr_osm.geojson")
smallpaths_without_gr_osm_segments = load_segments_from_geojson(graubunden_osm_path + "smallpaths_without_gr_osm.geojson")
smallpaths_with_gr_osm_segments = load_segments_from_geojson(graubunden_osm_path + "smallpaths_with_gr_osm.geojson")
print("Loaded OSM data for Graubunden")

Loaded Swiss Topo data for Zurich
Loaded Swiss Topo data for Graubunden
Loaded OSM data for Zurich
Loaded OSM data for Graubunden
